
# DisulfideLoader.ipynb
Demonstrating the ways to access Disulfide Bonds using the proteusPy
package.

Author: Eric G. Suchanek, PhD.

Last Modification: 2025-01-21 16:19:46

In [1]:
import proteusPy as pp

pp.configure_master_logger("loadertest.log")
_logger = pp.create_logger("__name__", "WARNING")
# pp.set_logger_level("loadertest", "WARNING")

Create an unfiltered ``DisulfideLoader`` with no distance constraints:

In [ ]:
pdb = pp.DisulfideLoader(verbose=True, subset=False, cutoff=-1, sg_cutoff=-1)

Access to the database is through the ``DisulfideLoader`` object. We can retrieve disulfides by PDB ID, integer indexing or slicing, disulfide name and by disulfide class. These will be illustrated below.

- Access by PDB ID:

In [ ]:
pdb["6dmb"]

- Access by numerical index:

In [ ]:
pdb[0]

- Access by slice:

In [ ]:
pdb[:5]

- Access by Disulfide Name:

In [ ]:
pdb["6dmb_203A_226A"]

- Access by Class Identifier, (specifiy the base explicity with a 'o' or 'b' suffix). If no suffix is included than base=8 is assumed, (to list the classes and their number of elements use the ``DisulfideLoader.print_classes()`` method with appropriate base:

In [ ]:
pdb.verbose = False
pdb["11212o"]

- Access by Class Identify without a suffix. Octant class is assumed

In [ ]:
pdb["11212"]

It's easy to display statistical information about the returned lists as follows:

- Show bond length and bond angle deviations for the first 1000 Disulfides:

In [ ]:
pdb[:1000].plot_deviation_histograms(theme="auto", log=True)

- Show the Cα-Cα and Sγ-Sγ distances for the first 1000 Disulfides:

In [ ]:
pdb[:1000].plot_distances(theme="auto", log=False, distance_type="ca")
pdb[:1000].plot_distances(theme="auto", log=True, distance_type="sg")

- Show the Ca-Ca distances for the entire database:

In [ ]:
pdb.plot_distances(theme="auto", log=True, distance_type="ca")

As you can see, the unfiltered database has a number of disulfides that exceed the maximum possible distance, (~8 A).

We can also display the torsion statistics in several ways:

- Statistics for a slice of disulfides:

In [ ]:
pdb[:10].display_torsion_statistics()

- Statistics for a class:
  - To list the binary classIDs use:

In [ ]:
pdb.get_class_df(base=2)

Since there are only 32 binary classes the above shows the overall distribution of Disulfides across ALL binary classes. The 8-fold (*octant*) class dataframe is *much* larger, (9697 members) and can be shown with:

```python
    pdb.get_class_df(base=8)
```


  Let's look at one of the binary classes (note: creating long lists of Disulfides takes time. The next cell takes over 12 seconds on my M3 Max MacBook Pro):

In [ ]:
pdb["02202b"].display_torsion_statistics(save=False, theme="auto")

The above shows quite large deviations for the dihedral angles. This suggests that the class is very broad in structural diversity. This is to be expected with a coarse structural filter, and was the driving reason to develop the *octant* dihedral angle quantization method.

In [ ]:
pdb["11212"].display_torsion_statistics(save=False, theme="auto")

- Statistics for a specific pdbID:

In [ ]:
pdb["6dmb"].display_torsion_statistics(theme="auto")

Finally, we can readily display either individual Disulfides or lists of them as follows:

In [17]:
best_ss = pdb["2q7q_75D_140D"]
worst_ss = pdb["6vxk_801B_806B"]
duo = pp.DisulfideList([best_ss, worst_ss], "bestworst")

In [ ]:
best_ss.display(style="sb")

We can display the list as multiple panels:

In [ ]:
duo.display(style="sb", light="dark")

Or we can display them overlaid onto a common coordinate frame:

In [ ]:
duo.display_overlay()